## 📋 COMPREHENSIVE STOCK ANALYSIS SUMMARY

In [ ]:
import yfinance as yf
import pandas as pd
import os
import glob
from datetime import datetime
import numpy as np

In [ ]:
path = os.getcwd()

In [ ]:
# Get all CSV files with 'stock_data' in filename
stock_files = glob.glob("*stock_data*.csv")

print("Stock data files:")
for file in stock_files:
    print(f"  {file}")

In [ ]:
file_name = "stock_data_current_20250916_nasdaq.csv"
file_path = path + "/" + file_name

In [ ]:
df = pd.read_csv(file_path, header = 0)
df = df.rename(columns={"Unnamed: 0": "Metrics"})
df = df.set_index('Metrics')

In [ ]:
# Calculate Price to Target Ratio (Current Price / Target Price) with proper float casting
df.loc['Price to Target Ratio'] = pd.to_numeric(df.loc['Current Price'], errors='coerce') / pd.to_numeric(df.loc['Target Price'], errors='coerce')

In [ ]:
# Let's first understand the complete dataset
print("📊 Dataset Overview:")
print(f"Total stocks: {len(df.columns)}")
print(f"Total metrics: {len(df.index)}")

In [ ]:
def filter_stocks_by_criteria(df, criteria):
    """
    Filter stocks based on multiple criteria
    
    Parameters:
    df: DataFrame from get_stock_metrics()
    criteria: dict of filtering criteria, e.g.:
        {
            'P/E Ratio TTM': {'max': 15, 'min': 1},
            'Current Ratio MRQ': {'min': 1.5},
            'ROE TTM': {'min': 0.15},
            'Debt to Equity MRQ': {'max': 50}
        }
    
    Returns:
    dict: {
        'filtered_tickers': list of tickers that meet all criteria,
        'results_summary': summary stats,
        'detailed_results': DataFrame with only filtered tickers
    }
    """
    filtered_tickers = []
    filter_results = {}
    
    for ticker in df.columns:
        meets_all_criteria = True
        ticker_results = {'ticker': ticker}
        
        # Check each criterion
        for metric, conditions in criteria.items():
            if metric not in df.index:
                ticker_results[f"{metric}_status"] = "metric_not_found"
                meets_all_criteria = False
                continue
                
            value = df.loc[metric, ticker]
            
            # Skip if no data (including NaN values)
            if pd.isna(value) or value == 'N/A' or value == 'NaN':
                ticker_results[f"{metric}_status"] = "no_data"
                meets_all_criteria = False
                continue
            
            # Convert to float if it's a string number
            try:
                if isinstance(value, str):
                    value = float(value)
            except (ValueError, TypeError):
                ticker_results[f"{metric}_status"] = "conversion_error"
                meets_all_criteria = False
                continue
            
            # Final check for numeric validity
            if not isinstance(value, (int, float)) or np.isnan(value):
                ticker_results[f"{metric}_status"] = "invalid_data"
                meets_all_criteria = False
                continue
                
            ticker_results[f"{metric}_value"] = value
            
            # Check min condition
            if 'min' in conditions and value < conditions['min']:
                ticker_results[f"{metric}_status"] = f"below_min_{conditions['min']}"
                meets_all_criteria = False
                continue
                
            # Check max condition  
            if 'max' in conditions and value > conditions['max']:
                ticker_results[f"{metric}_status"] = f"above_max_{conditions['max']}"
                meets_all_criteria = False
                continue
                
            ticker_results[f"{metric}_status"] = "pass"
        
        filter_results[ticker] = ticker_results
        
        if meets_all_criteria:
            filtered_tickers.append(ticker)
    
    # Create filtered DataFrame
    filtered_df = df[filtered_tickers] if filtered_tickers else pd.DataFrame()
    
    # Summary stats
    summary = {
        'total_tickers_analyzed': len(df.columns),
        'tickers_meeting_criteria': len(filtered_tickers),
        'success_rate': (len(filtered_tickers) / len(df.columns)) * 100 if df.columns.size > 0 else 0,
        'criteria_applied': criteria
    }
    
    return {
        'filtered_tickers': filtered_tickers,
        'results_summary': summary,
        'detailed_results': filtered_df,
        'filter_breakdown': filter_results
    }

In [ ]:
def comprehensive_stock_analysis(df):
    """
    Comprehensive stock analysis based on multiple financial health criteria
    
    Analysis Framework:
    
    [TO BE ADDED]
    
    Returns comprehensive analysis with tiered results
    """
    
    # Define criteria for different quality tiers
    excellent_criteria = {
        'P/E Ratio TTM': {'max': 18, 'min': 8},
        'P/B Ratio': {'max': 2.5, 'min': 0.5},
        'P/S Ratio TTM': {'max': 3, 'min': 0.2},
        'ROE TTM': {'min': 0.20},
        'Operating Margin MRQ': {'min': 0.18},
        'Profit Margin': {'min': 0.12},
        'Current Ratio MRQ': {'min': 2.0},
        'Debt to Equity MRQ': {'max': 40}
    }
    
    good_criteria = {
        'P/E Ratio TTM': {'max': 25, 'min': 6},
        'P/B Ratio': {'max': 4, 'min': 0.3},
        'P/S Ratio TTM': {'max': 5, 'min': 0.1},
        'ROE TTM': {'min': 0.15},
        'Operating Margin MRQ': {'min': 0.12},
        'Profit Margin': {'min': 0.08},
        'Current Ratio MRQ': {'min': 1.5},
        'Debt to Equity MRQ': {'max': 60}
    }
    
    acceptable_criteria = {
        'P/E Ratio TTM': {'max': 35, 'min': 4},
        'P/B Ratio': {'max': 6, 'min': 0.2},
        'P/S Ratio TTM': {'max': 8, 'min': 0.05},
        'ROE TTM': {'min': 0.10},
        'Current Ratio MRQ': {'min': 1.2},
        'Debt to Equity MRQ': {'max': 80}
    }
    
    momentum_criteria = {
      # Core financial (more relaxed)
      'ROE TTM': {'min': 0.10},                       
      'Current Ratio MRQ': {'min': 1.2},              
      'Debt to Equity MRQ': {'max': 60},              

      # More flexible valuation
      'P/E Ratio TTM': {'max': 35, 'min': 6},         
      'P/B Ratio': {'max': 3.5, 'min': 0.3},            

      # Very relaxed momentum (just basic filters)
      'Momentum 3M (%)': {'min': 5},
      'Momentum 1M (%)': {'min': 2},                
      'Price vs 50MA (%)': {'min': 0},
      'Price vs 200MA (%)': {'min': -5},              

      # Basic volume only
      'Avg Volume 3M': {'min': 300000},

      # Current price / Target price ratio < 1 (meaning current price < target price)
      'Price to Target Ratio': {'max': 0.9},
  } 
    
    # Apply filters
    excellent_stocks = filter_stocks_by_criteria(df, excellent_criteria)
    good_stocks = filter_stocks_by_criteria(df, good_criteria)  
    acceptable_stocks = filter_stocks_by_criteria(df, acceptable_criteria)
    momentum_stocks = filter_stocks_by_criteria(df, momentum_criteria)
    
    return {
        'excellent': excellent_stocks,
        'good': good_stocks,
        'acceptable': acceptable_stocks,
        'momentum': momentum_stocks,
        'criteria_explanation': {
            'excellent': 'Top-tier companies with exceptional financial metrics',
            'good': 'Solid companies with strong fundamentals', 
            'acceptable': 'Companies with adequate financial health',
            'momentum': 'Stocks showing strong recent performance and growth potential'
        }
    }

In [ ]:
# print("\n📋 ANALYSIS CRITERIA SUMMARY:")
# print("="*50)
# print("EXCELLENT TIER:")
# for metric, conditions in results['excellent']['results_summary']['criteria_applied'].items():
#     print(f"  • {metric}: {conditions}")

# print("\nGOOD TIER:")  
# for metric, conditions in results['good']['results_summary']['criteria_applied'].items():
#     print(f"  • {metric}: {conditions}")

# print("\nACCEPTABLE TIER:")
# for metric, conditions in results['acceptable']['results_summary']['criteria_applied'].items():
#     print(f"  • {metric}: {conditions}")
    
# print("\nMOMENTUM TIER:")
# for metric, conditions in results['momentum']['results_summary']['criteria_applied'].items():
#     print(f"  • {metric}: {conditions}")

In [ ]:
# Run comprehensive analysis
print("🔍 COMPREHENSIVE STOCK ANALYSIS")
print("="*50)

results = comprehensive_stock_analysis(df)

# Display results by tier
for tier in ['excellent', 'good', 'acceptable', 'momentum']: # took out accpetable for now
    tier_results = results[tier]
    print(f"\n🏆 {tier.upper()} STOCKS:")
    print(f"   {results['criteria_explanation'][tier]}")
    print(f"   Found: {len(tier_results['filtered_tickers'])} stocks")
    print(f"   Success Rate: {tier_results['results_summary']['success_rate']:.1f}%")
    
    if tier_results['filtered_tickers']:
        print("   Companies:")
        for ticker in tier_results['filtered_tickers']:
            company = df.loc['Company Name', ticker]
            sector = df.loc['Sector', ticker] if 'Sector' in df.index else 'N/A'
            current_price = df.loc['Current Price', ticker]
            pe_ratio = df.loc['P/E Ratio TTM', ticker]
            roe = df.loc['ROE TTM', ticker]
            print(f"     • {ticker}: {company}")
            print(f"       Sector: {sector} | Price: ${current_price} | P/E: {pe_ratio} | ROE: {roe}")
    else:
        print("   No stocks meet these criteria")
    print()

In [ ]:
monitor_ticker = 'MRK'

In [ ]:
pd.set_option("display.max_rows", None)
df.loc[:, monitor_ticker].to_frame()

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import math

def show_price_charts(ticker, periods=[7, 21, 90, 365]):
    """
    Display price charts for specified periods for a given ticker
    
    Parameters:
    ticker (str): Stock ticker symbol (e.g., 'AAPL', 'MSFT')
    periods (list): List of days to show charts for (default: [7, 21, 90, 365])
    """
    try:
        # Calculate grid layout (2 charts per row)
        n_charts = len(periods)
        n_rows = math.ceil(n_charts / 2)
        n_cols = min(2, n_charts)
        
        # Create figure with subplots
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 5 * n_rows))
        
        # Handle single chart case
        if n_charts == 1:
            axes = [axes]
        elif n_rows == 1:
            axes = [axes] if n_charts == 1 else axes
        else:
            axes = axes.flatten()
        
        # Get stock data
        stock = yf.Ticker(ticker)
        
        for i, days in enumerate(periods):
            # Calculate start date
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days + 10)  # Add buffer for weekends/holidays
            
            # Fetch data
            hist_data = stock.history(start=start_date, end=end_date)
            
            if hist_data.empty:
                axes[i].text(0.5, 0.5, f'No data available\nfor {ticker}', 
                           ha='center', va='center', transform=axes[i].transAxes)
                axes[i].set_title(f'{ticker} - Last {days} Days')
                continue
            
            # Get the last N days of data
            recent_data = hist_data.tail(days)
            
            # Plot the closing price
            axes[i].plot(recent_data.index, recent_data['Close'], 
                        linewidth=2, color='#1f77b4')
            axes[i].fill_between(recent_data.index, recent_data['Close'], 
                               alpha=0.3, color='#1f77b4')
            
            # Formatting
            axes[i].set_title(f'{ticker} - Last {days} Days')
            axes[i].set_xlabel('Date')
            axes[i].set_ylabel('Price ($)')
            axes[i].grid(True, alpha=0.3)
            axes[i].tick_params(axis='x', rotation=45)
            
            # Auto-scale y-axis to show fluctuations better
            if not recent_data.empty:
                price_min = recent_data['Close'].min()
                price_max = recent_data['Close'].max()
                price_range = price_max - price_min
                margin = price_range * 0.1  # Add 10% margin
                axes[i].set_ylim(price_min - margin, price_max + margin)
            
            # Add price info
            if not recent_data.empty:
                current_price = recent_data['Close'].iloc[-1]
                start_price = recent_data['Close'].iloc[0]
                change = current_price - start_price
                change_pct = (change / start_price) * 100
                
                color = 'green' if change >= 0 else 'red'
                axes[i].text(0.02, 0.98, 
                           f'Current: ${current_price:.2f}\n'
                           f'Change: ${change:.2f} ({change_pct:+.1f}%)',
                           transform=axes[i].transAxes, 
                           verticalalignment='top',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                           color=color, fontweight='bold')
        
        # Hide empty subplots if odd number of charts
        if n_charts % 2 == 1 and n_charts > 1:
            axes[-1].set_visible(False)
        
        plt.tight_layout()
        plt.show()
        
        # Print summary info
        print(f"\n📊 {ticker} Price Summary:")
        for days in periods:
            hist_data = stock.history(start=datetime.now() - timedelta(days=days + 10))
            if not hist_data.empty:
                recent_data = hist_data.tail(days)
                if not recent_data.empty:
                    current_price = recent_data['Close'].iloc[-1]
                    start_price = recent_data['Close'].iloc[0]
                    change = current_price - start_price
                    change_pct = (change / start_price) * 100
                    print(f"  • {days}D: ${current_price:.2f} ({change_pct:+.1f}%)")
        
    except Exception as e:
        print(f"❌ Error fetching data for {ticker}: {str(e)}")


In [ ]:
show_price_charts(monitor_ticker)
# show_price_charts(monitor_ticker, periods=[7, 21, 90, 365])  # Custom periods